# pybvh tutorial

## Bvh object

We will first load the library and create an object to see what is inside

In [15]:
import pybvh

bvhtest = pybvh.Bvh()
bvhtest

Bvh(nodes=[ROOT root], frames=array(shape=(1, 0), dtype=float64), frame_frequency=0.000000)

As can be seen, the 3 important parameters of a Bvh object are the followings: 

- nodes is a list of node object, either BvhNode for end site, BvhRoot for the root joint, or BvhJoint for the rest of the joints. It contains the bvh Hierarchy.
- frames is a 2D numpy array. Each line is a frame of the bvh animation. Each column is the rotational data for one axe of the euler angle of a given joint. This frames array is identical to the frame part of a bvh file.
- Finally the frame frequency tells us how fast the frames succeed each other, given by 1 / frames per second

### Loading a bvh file into a Bvh object

To better understand those parameters, let us load a bvh file into a bvh object.

In [19]:
from pathlib import Path

bvh_folder = Path('.')

filepath1 = bvh_folder / 'bvh_test1.bvh'
#filepath2 = bvh_folder / 'bvh_test2.bvh'

In [20]:
bvhtest1 = pybvh.read_bvh_file(filepath1)
bvhtest1

Bvh(nodes=[ROOT Hips, Spine, Spine1, Spine2, Spine3, Neck, Neck1, Head, RightShoulder, RightArm, RightForeArm, RightHand, LeftShoulder, LeftArm, LeftForeArm, LeftHand, RightUpLeg, RightLeg, RightFoot, RightToeBase, LeftUpLeg, LeftLeg, LeftFoot, LeftToeBase], frames=array(shape=(56, 75), dtype=float64), frame_frequency=0.008333)

We can see from this example the basic description of our skeleton, and that it has a total of 24 joints. The array contains 56 frames information. If you are wondering why we have 75 columns in the array instead of 72 (24joints*3angles), it is because the first 3 columns are the root position in the space, which add 3 columns, hence 75. Those frames are filmed at a rate of 1/0.00833 = 120 frames per second. Let us take a closer look at the hierarchy.

### Bvh nodes

In [23]:
bvhtest1.nodes[0:9]

[BvhRoot(name = Hips, offset = [0. 0. 0.], pos_channels = ['X', 'Y', 'Z'], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine)', 'BvhJoint(JOINT RightUpLeg)', 'BvhJoint(JOINT LeftUpLeg)'], parent = None),
 BvhJoint(name = Spine, offset = [-0.00000e+00  4.11045e+00  1.10000e-05], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine1)'], parent = ROOT Hips),
 BvhJoint(name = Spine1, offset = [ 2.000000e-06  4.048012e+00 -7.137590e-01], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine2)'], parent = JOINT Spine),
 BvhJoint(name = Spine2, offset = [ 1.00000e-06  4.09481e+00 -3.58239e-01], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine3)'], parent = JOINT Spine1),
 BvhJoint(name = Spine3, offset = [-0.00000e+00  4.11045e+00  1.10000e-05], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Neck)', 'BvhJoint(JOINT RightShoulder)', 'BvhJoint(JOINT LeftShoulder)'], parent = JOINT Spine2),
 BvhJoint(name = Neck, offset = [-1

Each element of this list is either
- a BvhRoot, only one, the root of the file; it is always the first element
- a BvhJoint, for all the actual joint other than the root
- a BvhNode, for the end point of some joints (the head in this example).



#### BvhNode

In [26]:
bvhtest1.nodes[8]

BvhNode(name = End Site Head, offset = [-0.00000e+00  3.40905e+00  9.00000e-06], parent = JOINT Head)

A BvhNode only consists of the name, an offset value and a pointer to its parent node in the nodes list. The offset value is alway [X, Y, Z]. It is normally used for the End site of the bvh files. If it is an End Site, it will alway say so in its name.|

The parent directly links to the actual object, which allow chaining operation:

In [28]:
bvhtest1.nodes[8].parent.name

'Head'

#### BvhJoint

In [30]:
bvhtest1.nodes[2]

BvhJoint(name = Spine1, offset = [ 2.000000e-06  4.048012e+00 -7.137590e-01], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine2)'], parent = JOINT Spine)

The BvhJoint inherits from the BvhNode class. In addition to the name, offset and parent parameters, it also has a rot_channels parameters. It indicates the Euler angle rotation order for this joint. It finally has a children parameter, which is a list of pointers to it children objects in the nodes list. Same as the parent, this allows acces to the object themselves.

In [32]:
print(bvhtest1.nodes[2].children[0].name, bvhtest1.nodes[2].children[0].offset)

Spine2 [ 1.00000e-06  4.09481e+00 -3.58239e-01]


#### BvhRoot

In [34]:
bvhtest1.nodes[0]

BvhRoot(name = Hips, offset = [0. 0. 0.], pos_channels = ['X', 'Y', 'Z'], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine)', 'BvhJoint(JOINT RightUpLeg)', 'BvhJoint(JOINT LeftUpLeg)'], parent = None)

The BvhRoot class inherits from the BvhJoint class. In addition to the previous parameters, it also defines a pos_channels parameters which tells us the order of the cartesian coordinate of the hips as they appear in the frames. This parameter can also be directly accessed through the parameter root.

In [36]:
bvhtest1.root

BvhRoot(name = Hips, offset = [0. 0. 0.], pos_channels = ['X', 'Y', 'Z'], rot_channels = ['Z', 'Y', 'X'], children = ['BvhJoint(JOINT Spine)', 'BvhJoint(JOINT RightUpLeg)', 'BvhJoint(JOINT LeftUpLeg)'], parent = None)

### Frames

The other very important parameter is the actual rotational data of each frames, given by a 2D numpy array. 

In [39]:
bvhtest1.frames

array([[ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  1.700000e-05,
        -2.000000e-06,  6.800499e+00],
       [ 1.020000e-02, -3.020000e-04,  4.990000e-04, ...,  1.700000e-05,
        -2.000000e-06,  6.827902e+00],
       [ 2.000000e-02, -1.000000e-03,  8.000000e-04, ...,  1.900000e-05,
        -2.000000e-06,  6.827901e+00],
       ...,
       [ 6.815990e-01, -4.340000e-02, -8.499000e-03, ...,  1.800000e-05,
        -2.000000e-06,  6.954298e+00],
       [ 6.927000e-01, -4.399900e-02, -7.499000e-03, ...,  1.800000e-05,
        -3.000000e-06,  6.964500e+00],
       [ 7.033000e-01, -4.470100e-02, -6.199000e-03, ...,  1.700000e-05,
        -2.000000e-06,  6.976403e+00]])

Since the relationship between column and joints/ax is not immediately apparent, we can get the parameter frame_template to help us with that

In [41]:
column_num = 8
print(f'the rotational data time series for {bvhtest1.frame_template[column_num]} is given by the 1D array \n{bvhtest1.frames[:,column_num]}')

the rotational data time series for Spine_X_rot is given by the 1D array 
[ 0.452089  0.439445  0.427663  0.41587   0.410912  0.401161  0.38933
  0.380368  0.37212   0.360267  0.35022   0.340748  0.334433  0.326398
  0.314985  0.303932  0.293721  0.285515  0.276949  0.268871  0.260183
  0.249918  0.234962  0.224661  0.218543  0.210785  0.203219  0.198842
  0.196693  0.187543  0.176877  0.167095  0.159633  0.142229  0.124546
  0.114552  0.102342  0.087777  0.067492  0.053082  0.04199   0.028276
  0.014381 -0.001682 -0.014426 -0.025318 -0.035848 -0.051094 -0.062776
 -0.076651 -0.088931 -0.104914 -0.116692 -0.130189 -0.137441 -0.148129]


## Bvh and DataFrame

### Bvh to DataFrame

Since this is stil not easy to see, we can use the pandas library to transform our 2D array into a DataFrame object. We use the method get_df_constructor() from the bvh object to facilitate this conversion. This conversion will also automatically add a time data at the front of the Dataframe.

In [45]:
import pandas as pd

df = pd.DataFrame(bvhtest1.get_df_constructor())
df.head()

,time,Hips_X_pos,Hips_Y_pos,Hips_Z_pos,Hips_Z_rot,Hips_Y_rot,Hips_X_rot,Spine_Z_rot,Spine_Y_rot,Spine_X_rot,...,LeftUpLeg_X_rot,LeftLeg_Z_rot,LeftLeg_Y_rot,LeftLeg_X_rot,LeftFoot_Z_rot,LeftFoot_Y_rot,LeftFoot_X_rot,LeftToeBase_Z_rot,LeftToeBase_Y_rot,LeftToeBase_X_rot
0,0.000000,0.0000,0.000000,0.000000,-31.367432,87.945325,-32.057233,-0.360491,1.071300,0.452089,...,2.880135,0.000013,0.000000,5.059800,0.520082,12.300448,-4.052205,0.000017,-0.000002,6.800499
1,0.008333,0.0102,-0.000302,0.000499,-31.628379,87.940263,-32.303246,-0.351126,1.058745,0.439445,...,2.881907,0.000013,-0.000001,5.062499,0.520025,12.310186,-4.068890,0.000017,-0.000002,6.827902
2,0.016667,0.0200,-0.001000,0.000800,-31.631200,87.934369,-32.292465,-0.350447,1.058266,0.427663,...,2.885708,0.000013,0.000000,5.063201,0.501073,12.318025,-4.086588,0.000019,-0.000002,6.827901
3,0.025000,0.0295,-0.001499,0.000600,-31.641838,87.933843,-32.289835,-0.349960,1.058789,0.415870,...,2.888723,0.000013,-0.000001,5.062901,0.494016,12.324353,-4.096336,0.000018,-0.000001,6.817102
4,0.033333,0.0395,-0.002499,0.000100,-31.596769,87.931473,-32.240026,-0.347546,1.059641,0.410912,...,2.902435,0.000013,-0.000001,5.063699,0.492816,12.330650,-4.115400,0.000018,-0.000002,6.824298


We can then work on the data as we normally would with a DataFrame.

In [47]:
df['Hips_X_pos'] = 0*df['Hips_X_pos']
df.head()

,time,Hips_X_pos,Hips_Y_pos,Hips_Z_pos,Hips_Z_rot,Hips_Y_rot,Hips_X_rot,Spine_Z_rot,Spine_Y_rot,Spine_X_rot,...,LeftUpLeg_X_rot,LeftLeg_Z_rot,LeftLeg_Y_rot,LeftLeg_X_rot,LeftFoot_Z_rot,LeftFoot_Y_rot,LeftFoot_X_rot,LeftToeBase_Z_rot,LeftToeBase_Y_rot,LeftToeBase_X_rot
0,0.000000,0.0,0.000000,0.000000,-31.367432,87.945325,-32.057233,-0.360491,1.071300,0.452089,...,2.880135,0.000013,0.000000,5.059800,0.520082,12.300448,-4.052205,0.000017,-0.000002,6.800499
1,0.008333,0.0,-0.000302,0.000499,-31.628379,87.940263,-32.303246,-0.351126,1.058745,0.439445,...,2.881907,0.000013,-0.000001,5.062499,0.520025,12.310186,-4.068890,0.000017,-0.000002,6.827902
2,0.016667,0.0,-0.001000,0.000800,-31.631200,87.934369,-32.292465,-0.350447,1.058266,0.427663,...,2.885708,0.000013,0.000000,5.063201,0.501073,12.318025,-4.086588,0.000019,-0.000002,6.827901
3,0.025000,0.0,-0.001499,0.000600,-31.641838,87.933843,-32.289835,-0.349960,1.058789,0.415870,...,2.888723,0.000013,-0.000001,5.062901,0.494016,12.324353,-4.096336,0.000018,-0.000001,6.817102
4,0.033333,0.0,-0.002499,0.000100,-31.596769,87.931473,-32.240026,-0.347546,1.059641,0.410912,...,2.902435,0.000013,-0.000001,5.063699,0.492816,12.330650,-4.115400,0.000018,-0.000002,6.824298


### DataFrame to Bvh

We can also do the opposite, and transform a DataFrame into a Bvh. However, since the Hierarchy information is not present in the DataFrame, we need another piece of information to complete it. There are two ways to create the Hierarchy information.

The first possibility is to have a list of Bvh nodes objects.

In [50]:
hierarchy_list = bvhtest1.nodes

We then call the df_to_bvh function to operate the magic. This function also do a deep copy of the hierarchy, so that the parent and child parameters points to the new objects and not the previous one.

In [52]:
new_bvh_object = pybvh.df_to_bvh(hierarchy_list, df)
new_bvh_object

Bvh(nodes=[ROOT Hips, Spine, Spine1, Spine2, Spine3, Neck, Neck1, Head, RightShoulder, RightArm, RightForeArm, RightHand, LeftShoulder, LeftArm, LeftForeArm, LeftHand, RightUpLeg, RightLeg, RightFoot, RightToeBase, LeftUpLeg, LeftLeg, LeftFoot, LeftToeBase], frames=array(shape=(56, 75), dtype=float64), frame_frequency=0.008333)

In [53]:
print(f"rotational data for the column '{new_bvh_object.frame_template[0]}' in our bvh object")
print(new_bvh_object.frames[0:5, 0:1])

rotational data for the column 'Hips_X_pos' in our bvh object
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


We see that indeed we saved the transformed data as a new Bvh object.

The other possibility to transfer the Hierarchy information is through a dictionnary of the folowing format:

We can write this dictionnary manually.The dictionnary is not assumed to be ordered so the position of the name of joint in the dictionnary is not taken into consideration. We can also obtain such a dictionnary through the Bvh object:

In [56]:
hier_info_dict = bvhtest1.hierarchy_info_as_dict()
print(f'The dictionnary keys:\n{hier_info_dict.keys()}')
print(f"\nExample of the content for the joint 'Hips':\n{hier_info_dict['Hips']}")

The dictionnary keys:
dict_keys(['Hips', 'Spine', 'Spine1', 'Spine2', 'Spine3', 'Neck', 'Neck1', 'Head', 'End Site Head', 'RightShoulder', 'RightArm', 'RightForeArm', 'RightHand', 'End Site RightHand', 'LeftShoulder', 'LeftArm', 'LeftForeArm', 'LeftHand', 'End Site LeftHand', 'RightUpLeg', 'RightLeg', 'RightFoot', 'RightToeBase', 'End Site RightToeBase', 'LeftUpLeg', 'LeftLeg', 'LeftFoot', 'LeftToeBase', 'End Site LeftToeBase'])

Example of the content for the joint 'Hips':
{'offset': array([0., 0., 0.]), 'pos_channels': ['X', 'Y', 'Z'], 'rot_channels': ['Z', 'Y', 'X'], 'children': ['Spine', 'RightUpLeg', 'LeftUpLeg'], 'parent': 'None'}


In [57]:
newer_bvh_object = pybvh.df_to_bvh(hier_info_dict, df)
newer_bvh_object

Bvh(nodes=[ROOT Hips, Spine, Spine1, Spine2, Spine3, Neck, Neck1, Head, RightShoulder, RightArm, RightForeArm, RightHand, LeftShoulder, LeftArm, LeftForeArm, LeftHand, RightUpLeg, RightLeg, RightFoot, RightToeBase, LeftUpLeg, LeftLeg, LeftFoot, LeftToeBase], frames=array(shape=(56, 75), dtype=float64), frame_frequency=0.008333)

## Saving a bvh file

Finally, if we want to save the changes, we can write a bvh file by simply giving a filepath to the Bvh class method to_bvh_file:

In [60]:
new_filepath = Path('./new_bvh.bvh')
newer_bvh_object.to_bvh_file(new_filepath)

Succesfully saved the file new_bvh.bvh at the location
C:\Users\victor\Desktop\RIEC\body_motion_ai\Basic Motion_BVH\pybvhProject
